<p style="font-size: 35px;">Trading Bot</p>
Algorithms:<br>
1. Buy low sell high algorithm

	-allow window for no trading between first 30min and last 30 minutes of the trading day<br>
	-needs to set what price the stock was bought at<br>
	-needs to sell right before hitting this price<br>
	-needs to sell 10% above what the stock was bought at (rough guess)<br>
	-3 variables: buy price, stop loss price, profit price = buy price + 10%<br>
</br>
      -Sell stocks at EOD

In [1]:
from dotenv import load_dotenv
from datetime import datetime
import json
load_dotenv()
import robin_stocks.robinhood as rs
import os
import requests

tock states<br>
Unbought<br>
Bought, hasnt met thresholdawd<br>
Bought, crossed threshold

In [2]:
#changing the sell option to not delete, deletion can be done at end of day because we can keep records on the price
#and weather its work buying again in the day.
import matplotlib.pyplot as plt
    
class Stock:
    def __init__(self, stockDict):
        for key, value in stockDict.items():
            setattr(self, key, value)
            
        # reset stock history
        self.history = []
        self.historySmoothed = []
        self.price_history=[]
        self.state= "UNBOUGHT"
        self.tick_count=0
        
        # NONE
        self.adaptiveSellFloor = None
        self.update(stockDict)
        
    def initiliaze(self):
        self.Price = float(self.ask_price)
        self.price_history.append(self.Price)

        
        
        
    def delete_self(self):
        # Perform any necessary cleanup or actions before deletion
        print("Deleting self...")
        
        # Delete the method from the object
        del self.delete_self

    def runFilter(self, adaptionInertia=0.8):
        # LPF / moving average for Buy Floor
        # only apply on upswing
        if self.bid_price_float > self.adaptiveSellFloor:
            self.adaptiveSellFloor = adaptionInertia*self.adaptiveSellFloor + self.bid_price_float * (1-adaptionInertia)
        self.historySmoothed += [self.adaptiveSellFloor]
        if len(self.historySmoothed) == 6:
            popself.historySmoothed.pop(0)
        
        
    def update(self, indict, adaptionInertia=0.8):
        
        for key, value in indict.items():
            exec("self." + key + " = value")
        print("updating " + self.symbol)
        self.bid_price_float = float(self.bid_price)
        self.history += [self.bid_price_float]
        
        # calculate the ratio to the buy time
        # ratioRelativeToBuy = float(self.bid_price) / float(self.buyPrice)
        
        # if the bid price falls below the floor, sell!
        if self.bid_price_float < self.adaptiveSellFloor:
            self.sell()
            
        self.runFilter()
            

    def graph(self):

        # Create the figure and axes
        fig, ax = plt.subplots()

        # Plot the first array
        ax.plot(self.historySmoothed, label='Smoothed')

        # Plot the second array
        ax.plot(self.history, label='Bid price')

        # Set the title
        ax.set_title('Stock History: ' + self.symbol)

        # Set the legend
        ax.legend()

        # Display the plot
        plt.show()

    
    def buy(self):
        self.buyPrice = float(self.ask_price)
        # set the adaptive sell floor to .95 of the buy price
        # only on the buy
        if self.adaptiveSellFloor is None:
            self.adaptiveSellFloor = 0.95 * self.buyPrice
        
        print("--------------------------------")
        print("Buying " + self.symbol)
        print("Bought at " + str(self.buyPrice))
        print("Adaptive Sell Foor " + str(self.adaptiveSellFloor))
        print("--------------------------------")
        
        self.state="BOUGHT"
        pass

    def sell(self):
        print("--------------------------------")
        print("Selling " + self.symbol)
        print("Bought at " + str(self.buyPrice))
        print("Sold at " + str(self.bid_price))
        print("YOUR $$$: " + str(self.bid_price_float - self.buyPrice))
        print("--------------------------------")
        
    
    # Tolerence Map

    #  
    #
    #   _________ buyPoint
    #
    #   _________ Adaptive sellFloor = LowPass Filter
    #
    #
    # AdaptionInertia is a parameter that lets us adjust the tracking speed of the LPF / Moving average\
            

In [3]:
dixonStocks = {}

In [4]:
def BuyLowSellHigh(stockListOfDicts):
    
    for stockDict in stockListOfDicts:
        stockSymbol = stockDict["symbol"]
        heldStockSymbols = dixonStocks.keys()
        
        # if we dont have the stock, buy it and add to dixonstocks
        # stock will buy itself upon creation
        if stockSymbol not in heldStockSymbols:
            dixonStocks[stockSymbol] = Stock(stockDict)
        # otherwise, run the update command, which may sell itself
        else:
            dixonStocks[stockSymbol].update(stockDict)
            
        

mport robin_stocks.robinhood as rh<br>
mport robin_stocks.gemini as gem<br>
mport robin_stocks.tda as tda

In [5]:
def get_top_100():
    top100listOfDicts = rs.get_top_100()
    top_100_data = top100listOfDicts.json
    top_100_file_path= "./Top_100.json"
    with open(top_100_file_path, "w+") as file:
        json.dump(top_100_data, file)
        print("updating Top_100.json")
    return(top100listOfDicts)

In [6]:
robin_user = os.environ.get("robinhood_username")
robin_pass = os.environ.get("robinhood_password")
rs.login(username=robin_user,
         password=robin_pass,
         expiresIn=86400,
         by_sms=True)

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJkY3QiOjE2ODY1MTg1MTcsImRldmljZV9oYXNoIjoiYWU2ZThhZGJhNTA2MDlhNDQ3NDg1M2U0YWI1ODM2M2QiLCJleHAiOjE2ODc5MDQxMjYsImxldmVsMl9hY2Nlc3MiOmZhbHNlLCJtZXRhIjp7Im9pZCI6ImM4MlNIMFdaT3NhYk9YR1Ayc3hxY2ozNEZ4a3ZmbldSWkJLbEJqRlMiLCJvbiI6IlJvYmluaG9vZCJ9LCJvcHRpb25zIjpmYWxzZSwic2NvcGUiOiJpbnRlcm5hbCIsInNlcnZpY2VfcmVjb3JkcyI6W3siaGFsdGVkIjpmYWxzZSwic2VydmljZSI6Im51bW11c191cyIsInNoYXJkX2lkIjoyLCJzdGF0ZSI6ImF2YWlsYWJsZSJ9LHsiaGFsdGVkIjpmYWxzZSwic2VydmljZSI6ImJyb2tlYmFja191cyIsInNoYXJkX2lkIjoxMSwic3RhdGUiOiJhdmFpbGFibGUifV0sInNybSI6eyJiX3VzIjp7ImhsIjpmYWxzZSwic2lkIjoxMSwic3QiOiJhdiJ9LCJuX3VzIjp7ImhsIjpmYWxzZSwic2lkIjoyLCJzdCI6ImF2In19LCJ0b2tlbiI6IkM4MUFoRTF2WHdCSXExeFJlc0lrb0w2dmMxUnZ3ayIsInVzZXJfaWQiOiIzNjExMGY5NC1jOWNhLTQ3NjItYmQ5Mi04MGRhNjYzYzU3MzIiLCJ1c2VyX29yaWdpbiI6IlVTIn0.qsrJ4fPB_2Rqtg5J7kzKwB6MXZ7251WYlDW0vx-uQIxhgu1lpgHLrzRzjYixnjzc2yMNCqAZYfOV1Nh4mMfhYkxf4icdYCJJs2rufPu5tsnZTc_gkGnYAxuAKR1-q_rUV2HD965JJ6OGlS6brnTKkOhgo5dtXGlVUrm4eAkM0XdPL

Get and print the symbols of the top 100 stocks

In [7]:
import json
import time
#print(dixonStocks)
while True:
    print("tick")
    # get top stocks
    #top100listOfDicts = rs.get_top_100()
    get_top_100()
    
    BuyLowSellHigh(top100listOfDicts)
    
    # get the 0th index in dixonStocks
    for k, stock in dixonStocks.items():
        stock.graph()
        break
    
    # every 5 seconds
    time.sleep(5)

tick


NameError: name 'top100listofDicts' is not defined

 = rs.get_all_stocks_from_market_tag('technology') # get all tech tags<br>
rint(a)

Here are some example calls

In [ ]:
rs.robinhood.get_all_open_crypto_orders() # gets all cypto orders from Robinhood
rs.logout()